<a href="https://colab.research.google.com/github/Requenamar3/Data-Mining/blob/main/MBA_shopify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running the Apriori Algorithm and Finding Association Rules in Online Retail Datasets


**Apply the Apriori algorithm and association rule mining to areal-world dataset**

##EDA

In [ ]:
!pip install mlxtend -qqq

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
 # installing the pandas_profiling package for data analysis and generating statistical report summaries.
!pip install ydata_profiling


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mlxtend.frequent_patterns
import mlxtend.preprocessing
import numpy
import datetime as dt
!pip install matplotlib


**check out the columns in the dataset in order to determine what columns to load**

In [ ]:
base_url = 'https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/'

# List of CSV file paths on GitHub
file_paths = [
    'sales_2023-01-01_2023-04-30.csv',
    'sales_2023-05-01_2023-09-30.csv',
    'sales_2024-03-07_2024-04-01.csv',
    'sales_2023-10-01_2024-03-06.csv',
]

# Function to get columns from a single CSV file
def get_columns_from_csv(file_path):
    full_url = f'{base_url}{file_path}'
    # Read just the first row to get the columns
    df = pd.read_csv(full_url, nrows=1)
    return df.columns.tolist()

# Check columns in each dataset
all_columns = set()
for file_path in file_paths:
    columns = get_columns_from_csv(file_path)
    all_columns.update(columns)

# print all_columns and decide which ones to load
print(all_columns)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'customer_cohort_quarter', 'total_sales', 'customer_type', 'product_title', 'customer_email', 'customer_cohort_month', 'order_name', 'month', 'product_type', 'shipping_postal_code', 'customer_cohort_week', 'units_per_transaction', 'product_price', 'shipping_region', 'total_cost', 'day', 'variant_sku', 'net_sales', 'customer_id', 'financial_status'}


**load only the columns needed**

In [ ]:
import pandas as pd

base_url = 'https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/'
file_paths = [
    'sales_2023-01-01_2023-04-30.csv',
    'sales_2023-05-01_2023-09-30.csv',
    'sales_2024-03-07_2024-04-01.csv',
    'sales_2023-10-01_2024-03-06.csv',
]

columns_to_load = [
    'total_sales', 'customer_type', 'product_title', 'customer_cohort_month',
    'order_name', 'month', 'shipping_postal_code', 'units_per_transaction',
    'product_price', 'shipping_region', 'total_cost', 'variant_sku',
    'net_sales', 'customer_id', 'financial_status'
]

dataframes_list = []

for file_path in file_paths:
    full_url = f'{base_url}{file_path}'
    # First, check which columns exist in the CSV
    available_columns = pd.read_csv(full_url, nrows=0).columns.tolist()
    # Determine the intersection of available columns and columns to load
    columns_to_read = [column for column in columns_to_load if column in available_columns]
    df = pd.read_csv(full_url, usecols=columns_to_read)
    dataframes_list.append(df)

BBORDERS = pd.concat(dataframes_list, ignore_index=True)
BBORDERS.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_id,customer_type,variant_sku,product_price,product_title,shipping_region,shipping_postal_code,customer_cohort_month,total_sales,net_sales,total_cost,units_per_transaction
0,2023-01,partially_refunded,#658025,5213564469363,Returning,SQ1726938,287.93,Bloomsy Deluxe,Texas,75204,2021-07,314.57,287.93,0.0,1
1,2023-01,refunded,#667324,5922255994995,Returning,NaN,0.00,NaN,Florida,34109,2022-07,41.94,0.00,0.0,0
2,2023-01,refunded,#667324,5922255994995,Returning,SQ9178530,269.94,Bloomsy Original,Florida,34109,2022-07,288.84,269.94,0.0,1
3,2023-01,refunded,#655100,5446435405939,Returning,SQ8862610,559.99,Bloomsy Premium BFCM 12 Months Prepaid,Indiana,47012,2021-12,0.00,0.00,0.0,0
4,2023-01,refunded,#655100,5446435405939,Returning,NaN,0.00,NaN,Indiana,47012,2021-12,0.00,0.00,0.0,0


In [ ]:
BBORDERS.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 184570 to 308208
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   financial_status       16 non-null     object        
 1   customer_id            16 non-null     int64         
 2   customer_type          16 non-null     object        
 3   product_price          16 non-null     float64       
 4   product_title          16 non-null     object        
 5   shipping_region        16 non-null     object        
 6   shipping_postal_code   16 non-null     object        
 7   total_sales            16 non-null     float64       
 8   net_sales              16 non-null     float64       
 9   total_cost             16 non-null     float64       
 10  units_per_transaction  16 non-null     int64         
 11  order_id               16 non-null     object        
 12  month_date             15 non-null     datetime64[ns]
 13  cus

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**create a unique Identifier by merging VariantSku and Ordername**

In [ ]:
BBORDERS=BBORDERS.dropna(subset=['variant_sku'])
BBORDERS['order_id']=BBORDERS['order_name']+'_'+BBORDERS['variant_sku']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-14-e1d7f84934f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BBORDERS['order_id']=BBORDERS['order_name']+'_'+BBORDERS['variant_sku']


In [ ]:
BBORDERS.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_id,customer_type,variant_sku,product_price,product_title,shipping_region,shipping_postal_code,customer_cohort_month,total_sales,net_sales,total_cost,units_per_transaction,order_id
0,2023-01,partially_refunded,#658025,5213564469363,Returning,SQ1726938,287.93,Bloomsy Deluxe,Texas,75204,2021-07,314.57,287.93,0.0,1,#658025_SQ1726938
2,2023-01,refunded,#667324,5922255994995,Returning,SQ9178530,269.94,Bloomsy Original,Florida,34109,2022-07,288.84,269.94,0.0,1,#667324_SQ9178530
3,2023-01,refunded,#655100,5446435405939,Returning,SQ8862610,559.99,Bloomsy Premium BFCM 12 Months Prepaid,Indiana,47012,2021-12,0.00,0.00,0.0,0,#655100_SQ8862610
5,2023-01,refunded,#667063,6444451954803,Returning,BLU697,99.99,Brighton,Texas,76086,2023-01,109.24,99.99,0.0,1,#667063_BLU697
7,2023-01,refunded,#666805,5513615736947,Returning,SQ8862610,324.99,Bloomsy Premium,Texas,77954,2022-01,0.00,0.00,0.0,0,#666805_SQ8862610


In [ ]:
#add new column and Convert 'month' to datetime format, assuming the first day of the month.
BBORDERS['month_date'] = pd.to_datetime(BBORDERS['month'], format='%Y-%m')

# add new column and Convert 'customer_cohort_month' to datetime format, assuming the first day of the month.
BBORDERS['customer_cohort_date'] = pd.to_datetime(BBORDERS['customer_cohort_month'], format='%Y-%m')

#remove unused columns
BBORDERS.drop(['month','variant_sku','order_name','customer_cohort_month'], axis=1, inplace=True)

BBORDERS.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,financial_status,customer_id,customer_type,product_price,product_title,shipping_region,shipping_postal_code,total_sales,net_sales,total_cost,units_per_transaction,order_id,month_date,customer_cohort_date
0,partially_refunded,5213564469363,Returning,287.93,Bloomsy Deluxe,Texas,75204,314.57,287.93,0.0,1,#658025_SQ1726938,2023-01-01,2021-07-01
2,refunded,5922255994995,Returning,269.94,Bloomsy Original,Florida,34109,288.84,269.94,0.0,1,#667324_SQ9178530,2023-01-01,2022-07-01
3,refunded,5446435405939,Returning,559.99,Bloomsy Premium BFCM 12 Months Prepaid,Indiana,47012,0.00,0.00,0.0,0,#655100_SQ8862610,2023-01-01,2021-12-01
5,refunded,6444451954803,Returning,99.99,Brighton,Texas,76086,109.24,99.99,0.0,1,#667063_BLU697,2023-01-01,2023-01-01
7,refunded,5513615736947,Returning,324.99,Bloomsy Premium,Texas,77954,0.00,0.00,0.0,0,#666805_SQ8862610,2023-01-01,2022-01-01


In [ ]:
#remove recods with 0 value
BBORDERS = BBORDERS.query('total_sales != 0 & total_cost != 0 & units_per_transaction != 0')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
BBORDERS.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 184570 to 308208
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   financial_status       16 non-null     object        
 1   customer_id            16 non-null     int64         
 2   customer_type          16 non-null     object        
 3   product_price          16 non-null     float64       
 4   product_title          16 non-null     object        
 5   shipping_region        16 non-null     object        
 6   shipping_postal_code   16 non-null     object        
 7   total_sales            16 non-null     float64       
 8   net_sales              16 non-null     float64       
 9   total_cost             16 non-null     float64       
 10  units_per_transaction  16 non-null     int64         
 11  order_id               16 non-null     object        
 12  month_date             15 non-null     datetime64[ns]
 13  cus

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**run the profiler to find duplicates and missing values**

In [ ]:
#Import the ProfileReport .For creating comprehensive exploratory data analysis reports.
from ydata_profiling import ProfileReport

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Create a ProfileReport object
profile = ProfileReport(BBORDERS, title="Shopify", explorative=True)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# display report
profile

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

##Part 1: Running the Apriori Algorithm on the Complete Online Retail Dataset

### 1.Run the Apriori Algorithm: Apply the Apriori algorithm to the full dataset. Set parameters to identify item sets appearing in at least 1% of transactions.


### 2.Analysis of Specific Item Set: Filter to the item set containing "10 COLOUR SPACEBOY PEN." Analyze its support value.

### 3.Item Set Length Analysis: Add a column for item set length. Focus on sets of length two with support between 0.02 and 0.021. Compare these to previous results.

### 4.Support Value Distribution: Plot the distribution of support values for the identified item sets

##Part 2: Finding the Association Rules on the Complete Online Retail Dataset

### 1.Fit Association Rule Model: Use the full dataset to generate association rules. Apply the confidence metric with a minimum threshold of 0.6.

### 2.Count and Compare Association Rules: Count the number of derived rules. Compare this with previous exercises.

### 3.Plot Confidence vs. Support: Create a plot to visualize the relationship between confidence and support for the rules.

### 4.Analyze Rule Metrics: Examine the distributions of confidence, lift, leverage, and conviction in your rules.

##Report writing



*   Insights gained from the Apriori algorithm and association rule mining.
*   How these insights can influence business decisions like store layout and pricing.

*   A comparison between the two datasets.
*   Reflections on how data partitioning and increasing data volume impact results.



